In [16]:
# ################
# Plot apsychrometric chart using psychrochart
# ################

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from psychrochart import PsychroChart
import pandas as pd
import psychrolib
import seaborn as sns
from datetime import datetime

mpl.rcParams['figure.dpi'] = 300

df = pd.read_csv('All_Sensors_Improved202302151432_Preventive Conserv.csv', sep=';')

format_string = "%d/%m/%Y %H:%M"

df['Date Time'] = df['Date Time'].apply(lambda x: datetime.strptime(x, format_string))

def convertir_en_float(chaine):
    chaine_modifiee = chaine.replace(',', '.')
    chaine_modifiee= chaine_modifiee.replace('-', 'nan')
    return float(chaine_modifiee)
df[df.columns[1:]] = df[df.columns[1:]].applymap(convertir_en_float)

# New column for the season
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

# Retrieve list of column names
df.columns = df.columns.str.replace('/', '-')
column_names = df.columns
## Dictionary to store the resulting DataFrames
dfs = {}

#  Loop to extract pairs of interesting columns
for i in range(54): #len(column_names)-1
    # Create a DataFrame with the average columns of the df
    selected_columns = df[[column_names[2*i+1], column_names[2*i+2]]] #can change depending on the input file
    # Add the resulting DataFrame to the dictionary
    dfs[(column_names[2*i+1], column_names[2*i+2])] = selected_columns.copy()

# Accéder aux DataFrames individuels
for key, selected_df in dfs.items():
    print(f"Columns: {key}")

    # Pass a dict with the changes wanted:
    custom_style = {
        "figure": {
            # "title": "Psychrometric Chart (sea level)",
            "x_label": "DRY-BULB TEMPERATURE, $°C$",
            "y_label": "HUMIDITY RATIO $g_w / kg_{da}$",
            "x_axis": {"color": [1.0, 1.0, 1.0], "linewidth": 1.5, "linestyle": "-"},
            "x_axis_labels": {"color": [1.0, 1.0, 1.0], "fontsize": 8},
            "x_axis_ticks": {"direction": "out", "color": [1.0, 1.0, 1.0]},
            "y_axis": {"color": [1.0, 1.0, 1.0], "linewidth": 1.5, "linestyle": "-"},
            "y_axis_labels": {"color": [1.0, 1.0, 1.0], "fontsize": 8},
            "y_axis_ticks": {"direction": "out", "color": [1.0, 1.0, 1.0]},
            "partial_axis": False,
            "position": [0.025, 0.075, 0.925, 0.875]
            },
        "limits": {
            "range_temp_c": [0, 40],
            "range_humidity_g_kg": [0, 15],
            "altitude_m": 0,
            "step_temp": 1.0
            },
        "saturation": {"color": [0.0, 0.0, 0.0], "linewidth": 1, "linestyle": "-"},
        "constant_rh": {"color": [0.8, 0.0, 0.0], "linewidth": 1, "linestyle": "-"},
        "constant_v": {"color": [0.0, 0.4, 0.4], "linewidth": 0.5, "linestyle": "-"},
        "constant_h": {"color": [1, 0.4, 0.0], "linewidth": 0.75, "linestyle": "-"},
        "constant_wet_temp": {"color": [0.2, 0.8, 0.2], "linewidth": 1, "linestyle": "--"},
        "constant_dry_temp": {"color": [0.2, 0.2, 0.2], "linewidth": 0.5, "linestyle": "-"},
        "constant_humidity": {"color": [0.2, 0.2, 0.2], "linewidth": 0.5, "linestyle": "-"},
        "chart_params": {
            "with_constant_rh": True,
            "constant_rh_curves": [20, 30, 40, 50, 60, 70, 80],
            "constant_rh_labels": [20, 40, 60], # 80],
            "with_constant_v": False,
            "constant_v_step": 0.01,
            "range_vol_m3_kg": [0.78, 0.96],
            "with_constant_h": False,
            "constant_h_step": 10,
            "constant_h_labels": [0],
            "range_h": [10, 130],
            "with_constant_wet_temp": False,
            "constant_wet_temp_step": 1,
            "range_wet_temp": [-10, 35],
            "constant_wet_temp_labels": [0, 5, 10, 15, 20], # 25, 30]
            "with_constant_dry_temp": True,
            "constant_temp_step": 5,

            "with_constant_humidity": True,
            "constant_humid_step": 2,

            "with_zones": False
            }
        }

    fig, ax = plt.subplots(figsize=(10, 10)) 
    chart = PsychroChart(custom_style) 
    chart.plot(ax)

# # Append zones:
    #zones_conf = {
    #    "zones":[{
    #            "zone_type": "dbt-rh",
    #            "style": {"edgecolor": [0.498, 0.624, 1.0], #0.498, 0.624, 0.8
    #                      "facecolor": [0.498, 0.624, 1.0, 0.2],
    #                      "linewidth": 2,
    #                      "linestyle": "-"},
    #            "points_x": [8, 16],
    #            "points_y": [30, 50],
    #            "label": "Cool zone"
    #        },
    #        {
    #            "zone_type": "dbt-rh",
    #            "style": {"edgecolor": [0.498, 0.624, 0.8],
    #                      "facecolor": [0.498, 0.624, 1.0, 0.2],
    #                      "linewidth": 2,
    #                      "linestyle": "--"},
    #            "points_x": [10.5, 13],
    #            "points_y": [75.5, 46],
    #            "label": "Winter"
            #}
#            ]}
#    chart.append_zones(zones_conf)
#    chart.plot(ax)

    pressure = 101325
    hr = []
    for ix, row in selected_df.iterrows():
        value1 = row[key[0]]
        value2 = row[key[1]]
        if value1 is not None and value2 is not None and value2 != 0:
            try:
                hr_value = psychrolib.GetHumRatioFromRelHum(value1, value2 / 100, pressure) * 1000
                hr.append(hr_value)
            except Exception as e:
                print(f"Error calculating HR for values: {value1}, {value2}")
                print(f"Error message: {str(e)}")
                hr.append(np.nan)
        else:
            hr.append(np.nan)
    selected_df[f"HR_{key[0]}_{key[1]}"] = hr
    selected_df["Date Time"] = df['Date Time']
    selected_df['Season'] = selected_df['Date Time'].dt.month.map(get_season)
    sns.histplot(selected_df, x=key[0], y=(f"HR_{key[0]}_{key[1]}"), ax=ax, hue='Season', binwidth=0.1)

    t_array = np.arange(0, 50, 0.1) #to have only white above 100% humidity
    lower_hr = [psychrolib.GetHumRatioFromRelHum(x, 1, pressure) * 1000 for x in t_array]
    ax.fill_between(t_array, lower_hr, 100, color='white')

    #plt.savefig(f"Psychro\psychro_{key[0][0:14]}.png", transparent=True)
    plt.close()



Columns: ('003_19_IPM (1120-00763) Chan: 1 C Average', '003_19_IPM (1120-00763) Chan: 2 %RH Average')


<ipython-input-16-19a6ab967bac>:150: RuntimeWarning: invalid value encountered in GetHumRatioFromRelHum
  hr_value = psychrolib.GetHumRatioFromRelHum(value1, value2 / 100, pressure) * 1000


Columns: ('009_14_C14 (1120-00758) Chan: 1 C Average', '009_14_C14 (1120-00758) Chan: 2 %RH Average')


<ipython-input-16-19a6ab967bac>:150: RuntimeWarning: invalid value encountered in GetHumRatioFromRelHum
  hr_value = psychrolib.GetHumRatioFromRelHum(value1, value2 / 100, pressure) * 1000


Columns: ('011_15_Serveur (1120-00759) Chan: 1 C Average', '011_15_Serveur (1120-00759) Chan: 2 %RH Average')
Columns: ('013_31_Rubens A (1120-00775) Chan: 1 C Average', '013_31_Rubens A (1120-00775) Chan: 2 %RH Average')
Columns: ('013_53_Salle Rubens B (1120-01214) Chan: 1 C Average', '013_53_Salle Rubens B (1120-01214) Chan: 2 %RH Average')
Columns: ('016_22_MiCaDas (1120-00766) Chan: 1 C Average', '016_22_MiCaDas (1120-00766) Chan: 2 %RH Average')
Columns: ('018_21_Sculpture pierre (1120-00765) Chan: 1 C Average', '018_21_Sculpture pierre (1120-00765) Chan: 2 %RH Average')
Columns: ('021_20_Peinture murale (1120-00764) Chan: 1 C Average', '021_20_Peinture murale (1120-00764) Chan: 2 %RH Average')


...

Columns: ('507_10_Reserve primitifs flamands (1120-00754) Chan: 1 C Average', '507_10_Reserve primitifs flamands (1120-00754) Chan: 2 %RH Average')


<ipython-input-16-19a6ab967bac>:150: RuntimeWarning: invalid value encountered in GetHumRatioFromRelHum
  hr_value = psychrolib.GetHumRatioFromRelHum(value1, value2 / 100, pressure) * 1000


Columns: ('528a_48_Textile C (1120-01199) Chan: 1 C Average', '528a_48_Textile C (1120-01199) Chan: 2 %RH Average')


<ipython-input-16-19a6ab967bac>:150: RuntimeWarning: invalid value encountered in GetHumRatioFromRelHum
  hr_value = psychrolib.GetHumRatioFromRelHum(value1, value2 / 100, pressure) * 1000


Columns: ('529a_50_Textile A (1120-01204) Chan: 1 C Average', '529a_50_Textile A (1120-01204) Chan: 2 %RH Average')
